In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt

import torch
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

2021-12-27 19:16:23.937736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib:/usr/local/lib/fst:/usr/local/lib:/usr/local/lib/fst:/usr/local/lib:/usr/local/lib/fst:/usr/local/lib:/usr/local/lib/fst:/usr/local/lib:/usr/local/lib/fst
2021-12-27 19:16:23.937751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df_train = pd.read_csv("train.tsv", sep="\t")
df_train['id'] = range(len(df_train))
df_train.set_index('id', inplace=True)
df_train.head()

,Relation,Sentence,NER1,NER2
id,,,,
0,occupation,कहानियों में यह सुना जाता है कि एक रहस्यमय तिब...,PERSON,OTHER
1,sport,లండన్: ఇంగ్లండ్ మాజీ కెప్టెన్ మైకేల్ వాన్ గురు...,EVENT,OTHER
2,sibling,"నగ్మా,<e2>జ్యోతిక</e2>,<e1>రోషిణి</e1> ముగ్గుర...",PERSON,PERSON
3,parent_organization,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...,ORG,ORG
4,mother,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...,PERSON,PERSON


In [3]:
df_en = pd.read_csv("en.tsv", sep="\t")
df_en['id'] = range(len(df_en))
df_en.set_index('id', inplace=True)
df_en.head()

,Relation,Sentence,NER1,NER2
id,,,,
0,director,<e1>The Battle of the Sexes</e1> is a 1928 Ame...,WORK_OF_ART,PERSON
1,director,<e1>The Day of Faith</e1> is a 1923 American s...,WORK_OF_ART,PERSON
2,director,<e1>The Straight Story</e1> is a 1999 biograph...,WORK_OF_ART,PERSON
3,director,"Baltz starred in ""<e2>Paul Thomas Anderson</e2...",WORK_OF_ART,PERSON
4,director,He also appeared in <e2>Clint Eastwood</e2>'s ...,WORK_OF_ART,PERSON


In [4]:
df_te = pd.read_csv("telugu_annotated_dump.csv", sep=",")
df_te = df_te.drop(columns=['Sentence','e1','e2', 'Unnamed: 0'])
df_te = df_te.rename(columns={'Sentence_tagged':'Sentence'})
df_te['id'] = range(len(df_te))
df_te.set_index('id', inplace=True)

df_te.head()

,Relation,NER1,NER2,Sentence
id,,,,
0,sport,EVENT,OTHER,Viral Video: <e2>క్రికెట్</e2> లో <e1>ఫీల్డింగ...
1,sport,EVENT,OTHER,ఇంటర్నెట్‌ డెస్క్‌: <e2>క్రికెట్</e2>లో అప్పుడ...
2,sport,EVENT,OTHER,ఇయాన్‌ బెల్‌ రనౌట్‌ వివాదాస్పదం.. యావత్ <e2>క్...
3,sport,EVENT,OTHER,ఇక వీడియో సంగతికి వస్తే.. ఒకే బంతికి మూడు సార్...
4,sport,EVENT,OTHER,ఐరోపా <e2>క్రికెట్</e2>‌ సిరీస్‌లో వర్మడో సీసీ...


In [5]:
df_be = pd.read_csv("bengali_annotated_dump.csv", sep=",")
df_be = df_be.drop(columns=['Sentence','e1','e2', 'Unnamed: 0'])
df_be = df_be.rename(columns={'Sentence_tagged':'Sentence'})
df_be['id'] = range(len(df_be))
df_be.set_index('id', inplace=True)

df_be.head()

,Relation,NER1,NER2,Sentence
id,,,,
0,parent_organization,ORG,ORG,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...
1,mother,PERSON,PERSON,মাতার গর্ভে থাকতেই তার শিক্ষা শুরু হয়েছিল গ...
2,mother,PERSON,PERSON,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...
3,mother,PERSON,PERSON,অর্জুনের ঔরসে দেবী <e2>সুভদ্রা</e2>র গর্ভে এক...
4,mother,PERSON,PERSON,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...


In [6]:
df_hi = pd.read_csv("hindi_annotated_dump.csv", sep=",")
df_hi = df_hi.drop(columns=['Sentence','e1','e2', 'Unnamed: 0'])
df_hi = df_hi.rename(columns={'Sentence_tagged':'Sentence'})
df_hi['id'] = range(len(df_hi))
df_hi.set_index('id', inplace=True)

df_hi.head()

,Relation,NER1,NER2,Sentence
id,,,,
0,occupation,PERSON,OTHER,<e1>मिलारेपा</e1> (तिब्बती: རྗེ་བཙུན་མི་ལ་རས་པ...
1,occupation,PERSON,OTHER,<e1>मिलारेपा</e1> (तिब्बती: རྗེ་བཙུན་མི་ལ་རས་པ...
2,occupation,PERSON,OTHER,12 वीं सदी में तिब्बती संत <e2>कवि</e2> ...
3,occupation,PERSON,OTHER,नई!!: <e1>मिलारेपा</e1> और <e2>कवि</e2> · और द...
4,occupation,PERSON,OTHER,कहानियों में यह सुना जाता है कि एक रहस्यमय ति...


In [7]:
df_eng = pd.read_csv("english_annotated_dump.csv", sep=",")
df_eng = df_eng.drop(columns=['Sentence','e1','e2', 'Unnamed: 0'])
df_eng = df_eng.rename(columns={'Sentence_tagged':'Sentence'})
df_eng['id'] = range(len(df_eng))
df_eng.set_index('id', inplace=True)

df_eng.head()

,Relation,NER1,NER2,Sentence
id,,,,
0,director,WORK_OF_ART,PERSON,<e1>The Battle of the Sexes</e1> is a 1914 Ame...
1,director,WORK_OF_ART,PERSON,<e1>The Battle of the Sexes</e1> is a 1928 Ame...
2,director,WORK_OF_ART,PERSON,<e1>The Day of Faith</e1> is a 1923 American s...
3,director,WORK_OF_ART,PERSON,<e1>The Straight Story</e1> is a 1999 biograph...
4,director,WORK_OF_ART,PERSON,"The website's critical consensus reads, ""With ..."


In [8]:
df = df_train.append(df_en).append(df_te).append(df_be).append(df_hi).append(df_eng)
df.head()

,Relation,Sentence,NER1,NER2
id,,,,
0,occupation,कहानियों में यह सुना जाता है कि एक रहस्यमय तिब...,PERSON,OTHER
1,sport,లండన్: ఇంగ్లండ్ మాజీ కెప్టెన్ మైకేల్ వాన్ గురు...,EVENT,OTHER
2,sibling,"నగ్మా,<e2>జ్యోతిక</e2>,<e1>రోషిణి</e1> ముగ్గుర...",PERSON,PERSON
3,parent_organization,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...,ORG,ORG
4,mother,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...,PERSON,PERSON


In [9]:
df.describe()

,Relation,Sentence,NER1,NER2
count,55428,55428,55428,55428
unique,25,48245,12,9
top,spouse,<e1>অহিংস যোগাযোগ</e1> বা ননভায়োলেন্ট কমিউনিক...,PERSON,PERSON
freq,5473,4,33464,28798


In [10]:
df.Relation.value_counts()

spouse                                  5473
director                                4776
child                                   4244
father                                  3937
sport                                   3205
capital                                 3194
mother                                  3065
position_held                           2607
founded_by                              2482
capital_of                              2461
sibling                                 2454
award_received                          2449
employer                                1729
original_language_of_film_or_TV_show    1695
parent_organization                     1653
occupation                              1610
tributary                               1389
subsidiary                              1199
participant                             1188
discoverer_or_inventor                   980
place_of_birth                           967
winner                                   894
place_of_d

In [11]:
df_nominated = df[df['Relation']=='nominated_for']

In [12]:
df = df.append([df_nominated]*2,ignore_index=True)

In [13]:
df.Relation.value_counts()

spouse                                  5473
director                                4776
child                                   4244
father                                  3937
sport                                   3205
capital                                 3194
mother                                  3065
position_held                           2607
founded_by                              2482
capital_of                              2461
sibling                                 2454
award_received                          2449
employer                                1729
original_language_of_film_or_TV_show    1695
parent_organization                     1653
occupation                              1610
tributary                               1389
subsidiary                              1199
participant                             1188
discoverer_or_inventor                   980
place_of_birth                           967
winner                                   894
place_of_d

In [14]:
df.describe()

,Relation,Sentence,NER1,NER2
count,55750,55750,55750,55750
unique,25,48245,12,9
top,spouse,"మదర్ థెరిస్సా, <e1>మార్టిన్ లూథర్ కింగ్</e1>, ...",PERSON,PERSON
freq,5473,6,33774,28802


In [15]:
# Convert relation labels to integer values
labels = df.Relation.unique()

label_dict = {label:index for index, label in enumerate(labels)}

In [16]:
df['Relation'] = df['Relation'].map(label_dict)
df.head()

,Relation,Sentence,NER1,NER2
0,0,कहानियों में यह सुना जाता है कि एक रहस्यमय तिब...,PERSON,OTHER
1,1,లండన్: ఇంగ్లండ్ మాజీ కెప్టెన్ మైకేల్ వాన్ గురు...,EVENT,OTHER
2,2,"నగ్మా,<e2>జ్యోతిక</e2>,<e1>రోషిణి</e1> ముగ్గుర...",PERSON,PERSON
3,3,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...,ORG,ORG
4,4,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...,PERSON,PERSON


In [17]:
df.head()

,Relation,Sentence,NER1,NER2
0,0,कहानियों में यह सुना जाता है कि एक रहस्यमय तिब...,PERSON,OTHER
1,1,లండన్: ఇంగ్లండ్ మాజీ కెప్టెన్ మైకేల్ వాన్ గురు...,EVENT,OTHER
2,2,"నగ్మా,<e2>జ్యోతిక</e2>,<e1>రోషిణి</e1> ముగ్గుర...",PERSON,PERSON
3,3,উরুগুয়ে জাতীয় ফুটবল দল (স্পেনীয়: Selección ...,ORG,ORG
4,4,<e1>অভিমন্যু</e1> মহাভারত মহাকাব্যের অন্যতম উল...,PERSON,PERSON


In [18]:
x=set(list(df['NER1'].unique())+ (list(df['NER2'].unique())))
x

{'DATE',
 'EVENT',
 'FAC',
 'GE',
 'GPE',
 'LANGUAGE',
 'LOC',
 'NORP',
 'ORG',
 'OTHER',
 'PERSON',
 'PRODUCT',
 'WORK_OF_ART'}

In [19]:
df['NER2'].unique()

array(['OTHER', 'PERSON', 'ORG', 'GPE', 'LOC', 'LANGUAGE', 'NORP',
       'WORK_OF_ART', 'EVENT'], dtype=object)

In [20]:
import re

In [21]:
df_test = df.copy()

In [22]:
df_test2=df_test.copy()

In [23]:
for i, row in df.iterrows():
    try:
        m = re.search('<e1>.*</e1>', row['Sentence']).span()
        t2=row['Sentence'][0:m[0]+4]+row['NER1']+row['Sentence'][m[1]-5:]
        m = re.search('<e2>.*</e2>', t2).span()
        df.at[i,'Sentence'] = t2[0:m[0]+4]+row['NER2']+t2[m[1]-5:]
    except:
        print(df.iloc[i].Sentence)
    

<e1>బాబర్</e1> కుమారుడు <e2>హుమాయూన్</e2> ఆగ్రాకు వచ్చినపుడు, విక్రమాదిత్య అంతఃపుర పరివారం పారిపోవడానికి ప్రయత్నించారు. అయితే హుమాయూన్ సైన్యం వారిని చుట్టుముట్టి కదలనీయలేదు. హుమాయూన్ వారిని దోచుకోవడానికి తన సైన్యాన్నీ అనుమతించలేదు. వారు ఇష్టపూర్వకంగా అనర్ఘరత్నాలు, ఆభరణాలు వున్న ఒక పేటికను హుమాయున్‌కు PERSON PERSON sibling సభా పర్వము -4 శిశుపాలుని వధ ఆ తరువాత ధర్మరాజు తన నలుగురు తమ్ములను నలుగురిని పిలిచి నాలుగు దిక్కులకు జైత్రయాత్రకు పంపాడు. నలుగురు తమ్ములు నాలుగు దిక్కుల ఉన్న రాజులను, దేశాలను జయించి కప్పం కట్టించుకుని తిరిగి వచ్చారు. <e2>భీమసేనుడు</e2>, <e1>అర్జునుడు</e1>, సహదేవుడు, నకులుడు నాలుగు దిక్కులను జయించి అసంఖ్యాకంగా ధన, కనక, వస్తు, వాహనములు తీసుకు వచ్చారు. ద్వారక నుండిశ్రీకృష్ణుడు అశేష ధన, కనక, వస్తు, వాహనాలతో వచ్చాడు. అవన్నీ ధర్మరాజుకు ఇచ్చి గౌరవించాడు…. PERSON PERSON founded_by গুগলের মূল কোম্পানি <e1>আলফাবেট ইনকর্পোরেটেড</e1> থেকে পদত্যাগের ঘোষণা দিয়েছেন প্রতিষ্ঠানটির দুই সহপ্রতিষ্ঠাতা <e2>ল্যারি পেইজ</e2> ও সের্গেই ব্রিন ORG PERSON tributary নামিবিয়ার ওরেভিস বে এবং দক্ষি

In [24]:
df.iloc[0].Sentence

'कहानियों में यह सुना जाता है कि एक रहस्यमय तिब्बती सिद्ध पुरुष और <e2>OTHER</e2> जिसका नाम <e1>PERSON</e1> था, वह एकमात्र इंसान थे जो कैलाश पर्वत की चोटी पर चढ़ने में सक्षम थे और यह कहानी भी 900 साल पुरानी मानी जाती है'

In [25]:
# df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DL4NLP/data/valid.tsv", names=['Relation','Sentence','NER1','NER2'], sep="\t")
# df_val['Relation'] = df_val['Relation'].map(label_dict)

# X_train, y_train = df.index.values, df.Relation.values
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.Relation.values, test_size=0.1, random_state=42)

In [26]:
# here they also add a label to indicate whether it is a training, validation or test set item.
# if you do cross-validation, you do not need such labeling.
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
# df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Relation', 'data_type']).count()

Sentence  NER1  NER2
Relation data_type                      
0        not_set         173   173   173
         train          1437  1437  1437
1        not_set         348   348   348
         train          2857  2857  2857
2        not_set         235   235   235
         train          2219  2219  2219
3        not_set         159   159   159
         train          1494  1494  1494
4        not_set         309   309   309
         train          2756  2756  2756
5        not_set         380   380   380
         train          3557  3557  3557
6        not_set         101   101   101
         train           866   866   866
7        not_set          77    77    77
         train           817   817   817
8        not_set          80    80    80
         train           702   702   702
9        not_set         178   178   178
         train          1551  1551  1551
10       not_set         524   524   524
         train          4949  4949  4949
11       not_set         247   247   247
         train          2235  2235  2235
12       not_set         399   399   399
         train          3845  3845  3845
13       not_set         322   322   322
         train          2872  2872  2872
14       not_set         483   483   483
         train          4293  4293  4293
15       not_set         145   145   145
         train          1244  1244  1244
16       not_set         125   125   125
         train          1074  1074  1074
17       not_set         153   153   153
         train          1542  1542  1542
18       not_set         129   129   129
         train          1059  1059  1059
19       not_set         252   252   252
         train          2197  2197  2197
20       not_set          92    92    92
         train           742   742   742
21       not_set         261   261   261
         train          2200  2200  2200
22       not_set         251   251   251
         train          2356  2356  2356
23       not_set         102   102   102
         train           878   878   878
24       not_set          50    50    50
         train           433   433   433

In [27]:
df.describe()

,Relation
count,55750.000000
mean,11.064323
std,6.574213
min,0.000000
25%,5.000000
50%,11.000000
75%,15.000000
max,24.000000


In [28]:
def add_special_tokens(self):
        special_tokens_dict = {'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>']}
        self.add_special_tokens(special_tokens_dict)
        self.model.resize_token_embeddings(len(self.tokenizer))

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [30]:
# special_tokens_dict = {'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>', 'DATE', 'EVENT', 'FAC', 'GE', 'GPE', 'LANGUAGE', 'LOC', 'NORP', 'ORG', 'OTHER', 'PERSON', 'PRODUCT', 'WORK_OF_ART']}
special_tokens_dict = {'additional_special_tokens': ['<e1>', '</e1>', '<e2>', '</e2>']}
tokenizer.add_special_tokens(special_tokens_dict)

4

In [31]:
df.Sentence = df.Sentence.apply(lambda x: "".join(['<s>', x, '</s>']))

In [32]:
encoded_data_train = tokenizer.batch_encode_plus(
    df.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df.Relation.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df.Relation.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/artus/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [33]:
encoded_data_val['input_ids'].shape

torch.Size([55750, 256])

In [34]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [35]:
len(label_dict)

25

In [38]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=len(label_dict), output_attentions=False, output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [36]:
from typing import Optional, Sequence

import torch.nn.functional as F
from torch import Tensor
import torch.nn as nn

softmax_layer = nn.LogSoftmax(dim=1)

# class FocalLoss(nn.modules.loss._WeightedLoss):
#     def __init__(self, weight=None, gamma=2, device='cpu'):
#         super(FocalLoss, self).__init__(weight)
#         # focusing hyper-parameter gamma
#         self.gamma = gamma

#         # class weights will act as the alpha parameter
#         self.weight = weight
        
#         # using deivce (cpu or gpu)
#         self.device = device
        
#         self.ce_loss = nn.CrossEntropyLoss()

#     def forward(self, _input, _target):
#         focal_loss = 0

#         for i in range(len(_input)):
#             # -log(pt)
#             cur_ce_loss = self.ce_loss(_input[i].view(-1, _input[i].size()[-1]), _target[i].view(-1))
#             # pt
#             pt = torch.exp(-cur_ce_loss)

#             if self.weight is not None:
#                 # alpha * (1-pt)^gamma * -log(pt)
#                 cur_focal_loss = self.weight[_target[i]] * ((1 - pt) ** self.gamma) * cur_ce_loss
#             else:
#                 # (1-pt)^gamma * -log(pt)
#                 cur_focal_loss = ((1 - pt) ** self.gamma) * cur_ce_loss
                
#             focal_loss = focal_loss + cur_focal_loss

#         if self.weight is not None:
#             focal_loss = focal_loss / self.weight.sum()
#             return focal_loss.to(self.device)
        
#         focal_loss = focal_loss / torch.tensor(len(probs))    
#         return focal_loss.to(self.device)

class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return 0.
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl
    
entropy_loss = focal_loss(alpha=None,gamma=2,device='cuda')

In [39]:
batch_size = 8

dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_val = DataLoader(dataset_val, sampler=RandomSampler(dataset_val), batch_size=8)

In [40]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [41]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

In [42]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [43]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [44]:
import random

seed_val = 123
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [46]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions = []
    true_vals = []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [47]:
model.resize_token_embeddings(len(tokenizer))

Embedding(105883, 768)

In [48]:
for epoch in tqdm(range(1,epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':        batch[0],
            'attention_mask':   batch[1],
            'labels':           batch[2],
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


Epoch 1
Training loss: 0.822655815018555



Validation loss: 0.4554372226275089
F1 Score (weighted): 0.8352116709297931



Epoch 2
Training loss: 0.41251936625219



Validation loss: 0.248915407111818
F1 Score (weighted): 0.9210985990202906



Epoch 3
Training loss: 0.30218870645183127



Validation loss: 0.17479398291068862
F1 Score (weighted): 0.9502611824163354



Epoch 4
Training loss: 0.23161996830719636



Validation loss: 0.12362116073078278
F1 Score (weighted): 0.9684376516033172



Epoch 5
Training loss: 0.1712596353893868



Validation loss: 0.10083237884161081
F1 Score (weighted): 0.9752378335075889



In [ ]:
accuracy_per_class(predictions, true_vals)

-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------

In [49]:
dfv = pd.read_csv("test.tsv", sep="\t", header=None)
dfv = dfv.rename(columns={0:'Id', 1:'Sentence',2:'NER1',3:'NER2'})
# dfv['id'] = range(len(dfv))
# dfv.set_index('Id', inplace=True)
dfv.head()

,Id,Sentence,NER1,NER2
0,id-0,খ্রি-মাং-স্লোন-র্ত্সানের মৃত্যুর পরেই ঝাংঝুং র...,PERSON,PERSON
1,id-1,<e1>John Stanislaus Joyce</e1> (4 July 1849 – ...,PERSON,PERSON
2,id-2,Dulquer Salman Telugu Movie: మలయాళ నటుడు <e1>మ...,PERSON,PERSON
3,id-3,<e1>Louise Taft</e1> would give birth to their...,PERSON,PERSON
4,id-4,(జన్మక్రమాన్ని బట్టి రాముడి తర్వాత భరతుడే. <e1...,PERSON,PERSON


In [50]:
# dfv.Sentence = dfv.Sentence.apply(lambda x: "".join(['<s>', x, '</s>']))
# for i, row in dfv.iterrows():
#     m = re.search('<e1>.*</e1>', row['Sentence']).span()
#     t2=row['Sentence'][0:m[1]]+' ['+row['NER1']+'] '+row['Sentence'][m[1]:]
#     m = re.search('<e2>.*</e2>', t2).span()
#     dfv.at[i,'Sentence'] = t2[0:m[1]]+' ['+row['NER2']+'] '+t2[m[1]:]

In [51]:
dfv.Sentence = dfv.Sentence.apply(lambda x: "".join(['<s>', x, '</s>']))
for i, row in dfv.iterrows():
    try:
        m = re.search('<e1>.*</e1>', row['Sentence']).span()
        t2=row['Sentence'][0:m[0]+4]+row['NER1']+row['Sentence'][m[1]-5:]
        m = re.search('<e2>.*</e2>', t2).span()
        dfv.at[i,'Sentence'] = t2[0:m[0]+4]+row['NER2']+t2[m[1]-5:]
    except:
        print(df.iloc[i].Sentence)
    

In [52]:
dfv.head()

,Id,Sentence,NER1,NER2
0,id-0,<s>খ্রি-মাং-স্লোন-র্ত্সানের মৃত্যুর পরেই ঝাংঝু...,PERSON,PERSON
1,id-1,<s><e1>PERSON</e1> (4 July 1849 – 29 December ...,PERSON,PERSON
2,id-2,<s>Dulquer Salman Telugu Movie: మలయాళ నటుడు <e...,PERSON,PERSON
3,id-3,<s><e1>PERSON</e1> would give birth to their s...,PERSON,PERSON
4,id-4,<s>(జన్మక్రమాన్ని బట్టి రాముడి తర్వాత భరతుడే. ...,PERSON,PERSON


In [53]:
encoded_data_val = tokenizer.batch_encode_plus(
    dfv.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']


dataset_val = TensorDataset(input_ids_val, attention_masks_val)
dataloader_val = DataLoader(dataset_val, sampler=None, batch_size=8)

/home/artus/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [54]:
def evaluate_val(dataloader_val):

    model.eval()
    predictions = []
  
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]                  
                 }

        with torch.no_grad():        
            outputs = model(**inputs)

        
        logits = outputs[0]
 

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    
    predictions = np.concatenate(predictions, axis=0)
            
    return predictions

In [55]:
preds = evaluate_val(dataloader_val)

In [56]:
preds[-15]

array([ 0.05162966,  0.17613932, -0.09245829, -0.10313057, -0.34614328,
       -1.2993044 ,  0.3027789 ,  0.37051564, -0.66244054,  0.32275167,
       -0.354475  , -0.46292016, -0.7982613 ,  0.09512513,  0.29456466,
        0.23195286,  0.20771267,  0.49972343, -0.44523588,  2.5242944 ,
       -0.2722284 , -0.23487647, -0.24484468, -1.7431875 ,  9.784786  ],
      dtype=float32)

In [57]:
preds2 = np.argmax(preds,axis=1)


In [58]:
label_dict_rev = dict((v,k) for k,v in label_dict.items())

In [59]:
label_dict_rev

{0: 'occupation',
 1: 'sport',
 2: 'sibling',
 3: 'parent_organization',
 4: 'mother',
 5: 'father',
 6: 'place_of_birth',
 7: 'winner',
 8: 'student_of',
 9: 'employer',
 10: 'spouse',
 11: 'founded_by',
 12: 'child',
 13: 'capital',
 14: 'director',
 15: 'tributary',
 16: 'subsidiary',
 17: 'original_language_of_film_or_TV_show',
 18: 'participant',
 19: 'award_received',
 20: 'place_of_death',
 21: 'capital_of',
 22: 'position_held',
 23: 'discoverer_or_inventor',
 24: 'nominated_for'}

In [60]:
dfv['preds'] = preds2

In [61]:
dfv['preds'] = dfv['preds'].map(label_dict_rev)

In [62]:
dfv

,Id,Sentence,NER1,NER2,preds
0,id-0,<s>খ্রি-মাং-স্লোন-র্ত্সানের মৃত্যুর পরেই ঝাংঝু...,PERSON,PERSON,child
1,id-1,<s><e1>PERSON</e1> (4 July 1849 – 29 December ...,PERSON,PERSON,child
2,id-2,<s>Dulquer Salman Telugu Movie: మలయాళ నటుడు <e...,PERSON,PERSON,child
3,id-3,<s><e1>PERSON</e1> would give birth to their s...,PERSON,PERSON,child
4,id-4,<s>(జన్మక్రమాన్ని బట్టి రాముడి తర్వాత భరతుడే. ...,PERSON,PERSON,spouse
...,...,...,...,...,...
3383,id-3383,<s>এছাড়া তিনি সিলভার লাইনিংস প্লেবুক চলচ্চিত্...,PERSON,OTHER,nominated_for
3384,id-3384,<s><e1>PERSON</e1> was twice nominated for the...,PERSON,OTHER,nominated_for
3385,id-3385,<s>He is best known for producing the critical...,PERSON,OTHER,award_received
3386,id-3386,"<s><e1>PERSON</e1> was critically acclaimed, a...",PERSON,OTHER,nominated_for


In [63]:
dfv.to_csv('df_bert_all_data_tmasktokens.csv')

dfv.to_csv('drive/MyDrive/df_bert_tel_en_train.csv')

In [ ]:
dfv = pd.read_csv("valid1.tsv", sep="\t", header=None)
dfv = dfv.rename(columns={0:'Id', 1:'Sentence',2:'NER1',3:'NER2'})
# dfv['id'] = range(len(dfv))
# dfv.set_index('Id', inplace=True)
dfv.head()

In [ ]:
dfv.Sentence = dfv.Sentence.apply(lambda x: "".join(['<s>', x, '</s>']))
for i, row in dfv.iterrows():
    m = re.search('<e1>.*</e1>', row['Sentence']).span()
    t2=row['Sentence'][0:m[1]]+' ['+row['NER1']+'] '+row['Sentence'][m[1]:]
    m = re.search('<e2>.*</e2>', t2).span()
    dfv.at[i,'Sentence'] = t2[0:m[1]]+' ['+row['NER2']+'] '+t2[m[1]:]

In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    dfv.Sentence.values,
    add_special_tokens=False,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']


dataset_val = TensorDataset(input_ids_val, attention_masks_val)
dataloader_val = DataLoader(dataset_val, sampler=None, batch_size=8)

In [ ]:
def evaluate_val(dataloader_val):

    model.eval()
    predictions = []
  
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]                  
                 }

        with torch.no_grad():        
            outputs = model(**inputs)

        
        logits = outputs[0]
 

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    
    predictions = np.concatenate(predictions, axis=0)
            
    return predictions

In [ ]:
preds = evaluate_val(dataloader_val)

In [ ]:
preds2 = np.argmax(preds,axis=1)


In [ ]:
label_dict_rev = dict((v,k) for k,v in label_dict.items())

In [ ]:
dfv['preds'] = preds2

In [ ]:
dfv['preds'] = dfv['preds'].map(label_dict_rev)

In [ ]:
dfv

In [ ]:
dfv.to_csv('df_bert_all_data_tokens_train_valid_dup_extratokens_new.csv')